In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
ruta_csv = "/content/drive/MyDrive/3 - Master Computacion y Sistemas Inteligentes/PFM/Datasets/dataset_empresa_03_pos_EDA_0.csv"
df = pd.read_csv(ruta_csv, low_memory=False)
df["con_fecha"] = pd.to_datetime(df["con_fecha"], errors="coerce")
df["con_dateticket"] = pd.to_datetime(df["con_dateticket"], errors="coerce")
df.info()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import itertools

anios = sorted(df['anio'].unique())
tipos = sorted(df['con_tipobillete'].unique())

combinaciones = pd.DataFrame(
    list(itertools.product(anios, tipos)),
    columns=['anio', 'con_tipobillete']
)

df_conteo = df.groupby(['anio', 'con_tipobillete'], as_index=False)['con_clave'].count()
df_conteo.columns = ['anio', 'con_tipobillete', 'num_registros']

df_completo = combinaciones.merge(df_conteo, on=['anio', 'con_tipobillete'], how='left')
df_completo['num_registros'] = df_completo['num_registros'].fillna(0).astype(int)

pivot = df_completo.pivot(index='anio', columns='con_tipobillete', values='num_registros').fillna(0).astype(int)

print("\n--- Numero de registros por tipo de billete y año ---")
display(pivot)

pivot.plot(
    kind='bar',
    stacked=False,
    figsize=(10,6),
    title='Distribucion de tipos de billete por año'
)
plt.xlabel('Año')
plt.ylabel('Numero de registros')
plt.legend(title='Tipo de billete', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
columnas_clave = [
    'con_clave',
    'con_fecha',
    'con_tipobillete',
    'con_naviera',
    'con_trayecto',
    'con_dateticket',
    'total_pasajeros',
    'total_vehiculos',
    'capacidad_pasajeros',
    'capacidad_vehiculos',
    'porc_ocupacion_pasajeros',
    'porc_ocupacion_vehiculos'
]
print("\n--- Valores nulos por columna clave ---")
print(df[columnas_clave].isnull().sum())
duplicados_exactos = df.duplicated().sum()
duplicados_por_clave = df.duplicated(subset=['con_clave']).sum()
print(f"\n--- Duplicados exactos: {duplicados_exactos}")
print(f"--- Duplicados por 'con_clave': {duplicados_por_clave}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')
df['con_dateticket'] = pd.to_datetime(df['con_dateticket'], errors='coerce')

df['anio_con_fecha'] = df['con_fecha'].dt.year
df['anio_con_dateticket'] = df['con_dateticket'].dt.year

conteo_con_fecha = df['anio_con_fecha'].value_counts().sort_index()
print("\n--- Conteo de registros por año (con con_fecha) ---")
print(conteo_con_fecha)

conteo_con_dateticket = df['anio_con_dateticket'].value_counts().sort_index()
print("\n--- Conteo de registros por año (con con_dateticket) ---")
print(conteo_con_dateticket)

plt.figure(figsize=(10,5))
plt.bar(conteo_con_fecha.index - 0.2, conteo_con_fecha.values, width=0.4, label='con_fecha')
plt.bar(conteo_con_dateticket.index + 0.2, conteo_con_dateticket.values, width=0.4, label='con_dateticket')

plt.title("Comparacion de distribucion por año: con_fecha vs con_dateticket")
plt.xlabel("Año")
plt.ylabel("Numero de registros")
plt.xticks(sorted(set(conteo_con_fecha.index).union(conteo_con_dateticket.index)))
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

columnas_tipologicas = [
    'con_naviera',
    'con_trayecto',
    'buq_nombre'
]

resultados = []

for col in columnas_tipologicas:
    n_original = df[col].nunique(dropna=False)
    n_transformado = df[col].astype(str).str.strip().str.upper().nunique(dropna=False)
    diferencia = n_original - n_transformado

    resultados.append({
        'columna': col,
        'valores unicos (original)': n_original,
        'valores unicos (normalizados)': n_transformado,
        'diferencia': diferencia
    })

df_tipologia_resumen = pd.DataFrame(resultados)

from IPython.display import display
display(df_tipologia_resumen)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_mes_billete = df.groupby(['anio', 'month', 'con_tipobillete']).size().reset_index(name='num_embarques')

anio_objetivo = 2024
df_anio = df_mes_billete[df_mes_billete['anio'] == anio_objetivo]

pivot = df_anio.pivot(index='month', columns='con_tipobillete', values='num_embarques').fillna(0)

pivot.plot(
    kind='line',
    marker='o',
    figsize=(10, 5),
    title=f'Distribucion mensual tipo de billete ({anio_objetivo})'
)
plt.xlabel('Mes')
plt.ylabel('Numero de embarques')
plt.xticks(range(1, 13))
plt.legend(title='Tipo de billete')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def clasificar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehiculo'
    else:
        return 'Otro'

df['tipo_billete_clasificado'] = df['con_tipobillete'].apply(clasificar_billete)

df_grupo = df.groupby(['anio', 'month', 'tipo_billete_clasificado']).size().reset_index(name='num_embarques')

anio_objetivo = 2023
df_anio = df_grupo[df_grupo['anio'] == anio_objetivo]

pivot = df_anio.pivot(index='month', columns='tipo_billete_clasificado', values='num_embarques').fillna(0)

pivot.plot(
    kind='line',
    marker='o',
    figsize=(10, 5),
    title=f'Distribucion mensual de embarques por tipo de categoría ({anio_objetivo})'
)
plt.xlabel('Mes')
plt.ylabel('Numero de embarques')
plt.xticks(range(1, 13))
plt.legend(title='Categoria')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def clasificar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehículo'
    else:
        return 'Otro'

df['tipo_billete_clasificado'] = df['con_tipobillete'].apply(clasificar_billete)

df_agrupado = df.groupby(['anio', 'month', 'tipo_billete_clasificado']).size().reset_index(name='num_embarques')

df_filtrado = df_agrupado[df_agrupado['tipo_billete_clasificado'].isin(['Pasajero', 'Vehículo'])]

fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
tipos = ['Pasajero', 'Vehiculo']
anios = sorted(df_filtrado['anio'].unique())
bar_width = 0.25
x = np.arange(1, 13)  

for i, tipo in enumerate(tipos):
    ax = axes[i]
    for j, anio in enumerate(anios):
        datos = df_filtrado[(df_filtrado['tipo_billete_clasificado'] == tipo) & (df_filtrado['anio'] == anio)]
        ax.bar(x + j * bar_width, datos['num_embarques'], width=bar_width, label=str(anio))

    ax.set_title(f'{tipo}s por mes y año')
    ax.set_xlabel('Mes')
    ax.set_xticks(x + bar_width) 
    ax.set_xticklabels(range(1, 13))
    ax.legend(title='Año')

axes[0].set_ylabel('Numero de embarques')
plt.suptitle('Comparativa mensual de embarques por agrupacion (pasajeros/vehiculos)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def clasificar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehículo'
    else:
        return 'Otro'

df['tipo_billete_clasificado'] = df['con_tipobillete'].apply(clasificar_billete)

df['fecha_mensual'] = df['con_fecha'].dt.to_period('M').dt.to_timestamp()

df_mensual = df.groupby(['fecha_mensual', 'tipo_billete_clasificado']).size().reset_index(name='num_embarques')

pivot = df_mensual.pivot(index='fecha_mensual', columns='tipo_billete_clasificado', values='num_embarques').fillna(0)

pivot.plot(figsize=(12,6), marker='o', title='Tendencia mensual de embarques por agrupacion')
plt.xlabel('Fecha')
plt.ylabel('Numero de embarques')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
df['nombre_dia'] = df['day_of_week'].map(dict(zip(range(7), dias_semana)))

df_dow = df.groupby(['nombre_dia', 'con_tipobillete']).size().reset_index(name='num_embarques')

orden_dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
df_dow['nombre_dia'] = pd.Categorical(df_dow['nombre_dia'], categories=orden_dias, ordered=True)

pivot = df_dow.pivot(index='nombre_dia', columns='con_tipobillete', values='num_embarques').fillna(0)

pivot.plot(kind='line', marker='o', figsize=(10, 5), title='Distribucion semanal de embarques por tipo de billete')
plt.xlabel('Dia de la semana')
plt.ylabel('Numero de embarques')
plt.xticks(rotation=45)
plt.legend(title='Tipo de billete')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_festivo = df.copy()
def clasificar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehiculo'
    else:
        return 'Otro'

df_festivo['tipo_billete_clasificado'] = df_festivo['con_tipobillete'].apply(clasificar_billete)

def tipo_festivo(row):
    if row['is_eid_aladha'] == 1:
        return 'Eid al-Adha'
    elif row['is_eid_alfitr'] == 1:
        return 'Eid al-Fitr'
    elif row['is_mawlid_nabi'] == 1:
        return 'Mawlid'
    elif row['is_festivo_nacional'] == 1:
        return 'Festivo Nacional'
    elif row['is_festivo_local'] == 1:
        return 'Festivo Local'
    else:
        return None  

df_festivo['tipo_festivo'] = df_festivo.apply(tipo_festivo, axis=1)

df_festivo = df_festivo[df_festivo['tipo_festivo'].notnull()]

df_grouped = df_festivo.groupby(['tipo_festivo', 'tipo_billete_clasificado']).size().reset_index(name='num_embarques')

pivot = df_grouped.pivot(index='tipo_festivo', columns='tipo_billete_clasificado', values='num_embarques').fillna(0)

pivot.plot(kind='bar', figsize=(10,6), title='Embarques durante festivos por categoria de billete')
plt.ylabel('Numero de embarques')
plt.xlabel('Tipo de festivo')
plt.xticks(rotation=45)
plt.legend(title='Categoria')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_rutas = df.copy()

def clasificar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehiculo'
    else:
        return 'Otro'

df_rutas['tipo_billete_clasificado'] = df_rutas['con_tipobillete'].apply(clasificar_billete)

rutas_objetivo = ['ESALGESCEU', 'ESALGMAPTM']
df_filtrado = df_rutas[df_rutas['con_trayecto'].isin(rutas_objetivo)]

df_grouped = df_filtrado.groupby(['anio', 'tipo_billete_clasificado', 'con_trayecto']).size().reset_index(name='num_embarques')

df_grouped['grupo'] = df_grouped['tipo_billete_clasificado'] + ' - ' + df_grouped['con_trayecto']

pivot = df_grouped.pivot(index='anio', columns='grupo', values='num_embarques').fillna(0)

pivot.plot(kind='bar', figsize=(10, 5))
plt.title('Comparativa anual de embarques por tipo de billete y ruta')
plt.xlabel('Año')
plt.ylabel('Nº Embarques')
plt.xticks(rotation=0)
plt.legend(title='Grupo (Categoría - Ruta)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_prop = df.copy()

def agrupar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehículo'
    else:
        return 'Otro'

df_prop['categoria_billete'] = df_prop['con_tipobillete'].apply(agrupar_billete)

df_prop['con_fecha'] = pd.to_datetime(df_prop['con_fecha'], errors='coerce')
df_prop['fecha_mensual'] = df_prop['con_fecha'].dt.to_period('M').dt.to_timestamp()

df_mes = df_prop.groupby(['fecha_mensual', 'categoria_billete']).size().reset_index(name='cuenta')

df_total_mes = df_prop.groupby('fecha_mensual').size().reset_index(name='total_mes')

df_merged = df_mes.merge(df_total_mes, on='fecha_mensual')
df_merged['porcentaje'] = (df_merged['cuenta'] / df_merged['total_mes']) * 100

pivot = df_merged.pivot(index='fecha_mensual', columns='categoria_billete', values='porcentaje')[['Pasajero', 'Vehículo']].fillna(0)

pivot.index = pivot.index.to_period('M').astype(str)
pivot.plot(kind='bar', figsize=(14, 5))
plt.title('Proporcion mensual de embarques por categoria')
plt.ylabel('Porcentaje (%)')
plt.xlabel('Mes')
plt.xticks(rotation=45)
plt.legend(title='Categoría')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')

df['fecha_sin_hora'] = df['con_fecha'].dt.date

eid_days = df.loc[df['is_eid_alfitr'] == 1, 'fecha_sin_hora'].drop_duplicates()
eid_days = sorted(eid_days)
print("Fechas con Eid al-Fitr:", eid_days)

todos_eids = []

for eid_day in eid_days:
    eid_day_dt = pd.to_datetime(eid_day) 
    start = eid_day_dt - pd.Timedelta(days=7)
    end = eid_day_dt + pd.Timedelta(days=7)
    df_window = df[(df['con_fecha'] >= start) & (df['con_fecha'] <= end)].copy()

    df_window['dias_relativos'] = (df_window['con_fecha'] - eid_day_dt).dt.days

    df_count = df_window.groupby('dias_relativos').size().reset_index(name='num_embarques')

    df_count['eid_date'] = eid_day_dt
    todos_eids.append(df_count)
    plt.figure(figsize=(8,5))
    plt.plot(df_count['dias_relativos'], df_count['num_embarques'], marker='o')
    plt.axvline(0, color='red', linestyle='--', label='Dia Eid')
    plt.title(f"Eid al-Fitr en {eid_day_dt.date()}: embarques 7 dias")
    plt.xlabel("Dias relativos (0 = dia de Eid)")
    plt.ylabel("Numero embarques")
    plt.legend()
    plt.tight_layout()
    plt.show()

df_concat = pd.concat(todos_eids, ignore_index=True)
df_avg = df_concat.groupby('dias_relativos')['num_embarques'].mean().reset_index()

plt.figure(figsize=(8,5))
plt.plot(df_avg['dias_relativos'], df_avg['num_embarques'], marker='o')
plt.axvline(0, color='red', linestyle='--', label='Dia Eid')
plt.title("Promedio embarques 7 dias a Eid al-Fitr (todos los años)")
plt.xlabel("(0 = dia de Eid)")
plt.ylabel("Promedio embarques")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')
df['fecha_sin_hora'] = df['con_fecha'].dt.date

eid_aladha_days = df.loc[df['is_eid_aladha'] == 1, 'fecha_sin_hora'].drop_duplicates()
eid_aladha_days = sorted(eid_aladha_days)
print("Fechas con Eid al-Adha:", eid_aladha_days)

todos_eids_aladha = []

for eid_day in eid_aladha_days:
    eid_day_dt = pd.to_datetime(eid_day) 
    start = eid_day_dt - pd.Timedelta(days=7)
    end = eid_day_dt + pd.Timedelta(days=7)

    df_window = df[(df['con_fecha'] >= start) & (df['con_fecha'] <= end)].copy()

    df_window['dias_relativos'] = (df_window['con_fecha'] - eid_day_dt).dt.days

    df_count = df_window.groupby('dias_relativos').size().reset_index(name='num_embarques')

    df_count['eid_date'] = eid_day_dt
    todos_eids_aladha.append(df_count)

    plt.figure(figsize=(8,5))
    plt.plot(df_count['dias_relativos'], df_count['num_embarques'], marker='o')
    plt.axvline(0, color='red', linestyle='--', label='Dia Eid')
    plt.title(f"Eid al-Adha en {eid_day_dt.date()}: embarques 7 dias")
    plt.xlabel("Dias relativos (0 = dia de Eid)")
    plt.ylabel("Nº embarques")
    plt.legend()
    plt.tight_layout()
    plt.show()

df_concat_aladha = pd.concat(todos_eids_aladha, ignore_index=True)
df_avg_aladha = df_concat_aladha.groupby('dias_relativos')['num_embarques'].mean().reset_index()

plt.figure(figsize=(8,5))
plt.plot(df_avg_aladha['dias_relativos'], df_avg_aladha['num_embarques'], marker='o')
plt.axvline(0, color='red', linestyle='--', label='Día Eid')
plt.title("Promedio embarques 7 días a Eid al-Adha (todos los años)")
plt.xlabel("(0 = dia de Eid)")
plt.ylabel("Promedio de embarques")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')
df['fecha_sin_hora'] = df['con_fecha'].dt.date

def clasificar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehículo'
    else:
        return 'Otro'

eid_aladha_days = df.loc[df['is_eid_aladha'] == 1, 'fecha_sin_hora'].drop_duplicates()
eid_aladha_days = sorted(eid_aladha_days)
print("Fechas con Eid al-Adha:", eid_aladha_days)

todos_eids_aladha = []

for eid_day in eid_aladha_days:
    eid_day_dt = pd.to_datetime(eid_day)
    start = eid_day_dt - pd.Timedelta(days=7)
    end = eid_day_dt + pd.Timedelta(days=7)

    df_window = df[(df['con_fecha'] >= start) & (df['con_fecha'] <= end)].copy()

    df_window['dias_relativos'] = (df_window['con_fecha'] - eid_day_dt).dt.days

    df_window['categoria'] = df_window['con_tipobillete'].apply(clasificar_billete)

    df_count = df_window.groupby(['dias_relativos', 'categoria']).size().reset_index(name='num_embarques')
    df_count['eid_date'] = eid_day_dt
    todos_eids_aladha.append(df_count)

    pivot = df_count.pivot(index='dias_relativos', columns='categoria', values='num_embarques').fillna(0)
    plt.figure(figsize=(8,5))
    for cat in pivot.columns:
        plt.plot(pivot.index, pivot[cat], marker='o', label=cat)
    plt.axvline(0, color='red', linestyle='--', label='Día Eid')
    plt.title(f"Eid al-Adha en {eid_day_dt.date()}: embarques ±7 días")
    plt.xlabel("Dias relativos (0 = día de Eid)")
    plt.ylabel("Numero de embarques")
    plt.legend()
    plt.tight_layout()
    plt.show()

df_concat_aladha = pd.concat(todos_eids_aladha, ignore_index=True)
df_avg_aladha = df_concat_aladha.groupby(['dias_relativos', 'categoria'])['num_embarques'].mean().reset_index()
pivot_avg = df_avg_aladha.pivot(index='dias_relativos', columns='categoria', values='num_embarques').fillna(0)
plt.figure(figsize=(8,5))
for cat in pivot_avg.columns:
    plt.plot(pivot_avg.index, pivot_avg[cat], marker='o', label=cat)
plt.axvline(0, color='red', linestyle='--', label='Día Eid')
plt.title("Promedio de embarques 7 días en torno a Eid al-Adha (por categoría)")
plt.xlabel("Dias relativos (0 = día de Eid)")
plt.ylabel("Numero promedio de embarques")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')
df['fecha_sin_hora'] = df['con_fecha'].dt.date

def clasificar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehículo'
    else:
        return 'Otro'

eid_aladha_days = df.loc[df['is_eid_aladha'] == 1, 'fecha_sin_hora'].drop_duplicates()
eid_aladha_days = sorted(eid_aladha_days)
print("Fechas con Eid al-Adha:", eid_aladha_days)

todos_eids_aladha = []

plt.figure(figsize=(12,6))
for eid_day in eid_aladha_days:
    eid_day_dt = pd.to_datetime(eid_day) 
    start = eid_day_dt - pd.Timedelta(days=7)
    end = eid_day_dt + pd.Timedelta(days=7)

    df_window = df[(df['con_fecha'] >= start) & (df['con_fecha'] <= end)].copy()

    df_window['dias_relativos'] = (df_window['con_fecha'] - eid_day_dt).dt.days

    df_window['categoria'] = df_window['con_tipobillete'].apply(clasificar_billete)

    df_count = df_window.groupby(['dias_relativos', 'categoria']).size().reset_index(name='num_embarques')
    df_count['eid_date'] = eid_day_dt
    todos_eids_aladha.append(df_count)

    pivot = df_count.pivot(index='dias_relativos', columns='categoria', values='num_embarques').fillna(0)

    for cat in pivot.columns:
        plt.plot(pivot.index, pivot[cat], marker='o', linestyle='-', label=f"{eid_day_dt.date()} - {cat}")

plt.axvline(0, color='red', linestyle='--', label='Día Eid')
plt.title("Evolucion diaria de embarques en torno a Eid al-Adha (individual por año y categoria)")
plt.xlabel("Dias relativos (0 = dia de Eid)")
plt.ylabel("Numero de embarques")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.tight_layout()
plt.show()

df_concat_aladha = pd.concat(todos_eids_aladha, ignore_index=True)
df_avg_aladha = df_concat_aladha.groupby(['dias_relativos', 'categoria'])['num_embarques'].mean().reset_index()
pivot_avg = df_avg_aladha.pivot(index='dias_relativos', columns='categoria', values='num_embarques').fillna(0)

plt.figure(figsize=(12,6))
for cat in pivot_avg.columns:
    plt.plot(pivot_avg.index, pivot_avg[cat], marker='o', linestyle='-', label=cat)
plt.axvline(0, color='red', linestyle='--', label='Día Eid')
plt.title("Promedio global de embarques 7 días en torno a Eid al-Adha (por categoría)")
plt.xlabel("Dias relativos (0 = dia de Eid)")
plt.ylabel("Numero promedio de embarques")
plt.legend(title='Categoría')
plt.tight_layout()
plt.show()




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')
df['fecha_sin_hora'] = df['con_fecha'].dt.date

def clasificar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehículo'
    else:
        return 'Otro'

eid_aladha_days = df.loc[df['is_eid_aladha'] == 1, 'fecha_sin_hora'].drop_duplicates()
eid_aladha_days = sorted(eid_aladha_days)

todos_eids_aladha = []

for eid_day in eid_aladha_days:
    eid_day_dt = pd.to_datetime(eid_day)
    start = eid_day_dt - pd.Timedelta(days=7)
    end = eid_day_dt + pd.Timedelta(days=7)

    df_window = df[(df['con_fecha'] >= start) & (df['con_fecha'] <= end)].copy()
    df_window['dias_relativos'] = (df_window['con_fecha'] - eid_day_dt).dt.days
    df_window['categoria'] = df_window['con_tipobillete'].apply(clasificar_billete)

    df_count = df_window.groupby(['dias_relativos','categoria']).size().reset_index(name='num_embarques')
    df_count['eid_date'] = eid_day_dt
    todos_eids_aladha.append(df_count)

df_concat_aladha = pd.concat(todos_eids_aladha, ignore_index=True)

df_avg_global = df_concat_aladha.groupby(['dias_relativos','categoria'])['num_embarques'].mean().reset_index()
pivot_global = df_avg_global.pivot(index='dias_relativos', columns='categoria', values='num_embarques').fillna(0)

plt.figure(figsize=(10,6))
for cat in pivot_global.columns:
    plt.plot(pivot_global.index, pivot_global[cat], marker='o', label=cat)
plt.axvline(0, color='red', linestyle='--', label='Dia Eid')
plt.title("Promedio global embarques 7 dias a Eid al-Adha")
plt.xlabel("(0 = dia de Eid)")
plt.ylabel("Promedio embarques")
plt.legend(title='Categoria')
plt.tight_layout()
plt.show()

df_2024 = df_concat_aladha[df_concat_aladha['eid_date'].dt.year == 2024]

df_avg_2024 = df_2024.groupby(['dias_relativos','categoria'])['num_embarques'].mean().reset_index()
pivot_2024 = df_avg_2024.pivot(index='dias_relativos', columns='categoria', values='num_embarques').fillna(0)

plt.figure(figsize=(10,6))
for cat in pivot_2024.columns:
    plt.plot(pivot_2024.index, pivot_2024[cat], marker='o', label=cat)
plt.axvline(0, color='red', linestyle='--', label='Día Eid')
plt.title("Promedio embarques 7 dias a Eid al-Adha (2024)")
plt.xlabel("(0 = dia de Eid)")
plt.ylabel("Promedio de embarques")
plt.legend(title='Categoria')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10,6))

for cat in pivot_global.columns:
    plt.plot(
        pivot_global.index,
        pivot_global[cat],
        marker='o',
        linestyle='-',
        label=f"Global - {cat}"
    )

for cat in pivot_2024.columns:
    plt.plot(
        pivot_2024.index,
        pivot_2024[cat],
        marker='o',
        linestyle='--',
        label=f"2024 - {cat}"
    )

plt.axvline(0, color='red', linestyle='--', label='Día Eid')
plt.title("Comparacion: Promedio global vs. Promedio 2024 (Eid al-Adha)")
plt.xlabel("Dias relativos (0 = dia de Eid)")
plt.ylabel("Numero promedio de embarques")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')
df['fecha_sin_hora'] = df['con_fecha'].dt.date

def clasificar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehículo'
    else:
        return 'Otro'

eid_alfitr_days = df.loc[df['is_eid_alfitr'] == 1, 'fecha_sin_hora'].drop_duplicates()
eid_alfitr_days = sorted(eid_alfitr_days)

todos_eids_alfitr = []

for eid_day in eid_alfitr_days:
    eid_day_dt = pd.to_datetime(eid_day)
    start = eid_day_dt - pd.Timedelta(days=7)
    end = eid_day_dt + pd.Timedelta(days=7)

    df_window = df[(df['con_fecha'] >= start) & (df['con_fecha'] <= end)].copy()
    df_window['dias_relativos'] = (df_window['con_fecha'] - eid_day_dt).dt.days
    df_window['categoria'] = df_window['con_tipobillete'].apply(clasificar_billete)

    df_count = df_window.groupby(['dias_relativos','categoria']).size().reset_index(name='num_embarques')
    df_count['eid_date'] = eid_day_dt
    todos_eids_alfitr.append(df_count)

df_concat_alfitr = pd.concat(todos_eids_alfitr, ignore_index=True)

df_avg_global = df_concat_alfitr.groupby(['dias_relativos','categoria'])['num_embarques'].mean().reset_index()
pivot_global = df_avg_global.pivot(index='dias_relativos', columns='categoria', values='num_embarques').fillna(0)

plt.figure(figsize=(10,6))
for cat in pivot_global.columns:
    plt.plot(pivot_global.index, pivot_global[cat], marker='o', label=f"Global - {cat}")
plt.axvline(0, color='red', linestyle='--', label='Dia Eid')
plt.title("Promedio global embarques 7 dias a Eid al-Fitr")
plt.xlabel("Dias relativos (0 = dia de Eid)")
plt.ylabel("Promedio embarques")
plt.legend(title='Categoria')
plt.tight_layout()
plt.show()

df_2024 = df_concat_alfitr[df_concat_alfitr['eid_date'].dt.year == 2024]
df_avg_2024 = df_2024.groupby(['dias_relativos','categoria'])['num_embarques'].mean().reset_index()
pivot_2024 = df_avg_2024.pivot(index='dias_relativos', columns='categoria', values='num_embarques').fillna(0)

plt.figure(figsize=(10,6))
for cat in pivot_2024.columns:
    plt.plot(pivot_2024.index, pivot_2024[cat], marker='o', label=f"2024 - {cat}")
plt.axvline(0, color='red', linestyle='--', label='Dia Eid')
plt.title("Promedio embarques 7 dias a Eid al-Fitr (2024)")
plt.xlabel("Dias relativos (0 = dia de Eid)")
plt.ylabel("Promedio embarques")
plt.legend(title='Categoria')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,6))
for cat in pivot_global.columns:
    plt.plot(pivot_global.index, pivot_global[cat], marker='o', linestyle='-', label=f"Global - {cat}")
for cat in pivot_2024.columns:
    plt.plot(pivot_2024.index, pivot_2024[cat], marker='o', linestyle='--', label=f"2024 - {cat}")
plt.axvline(0, color='red', linestyle='--', label='Dia Eid')
plt.title("Comparacion global vs 2024 - Eid al-Fitr (7 dias)")
plt.xlabel("Dias relativos (0 = dia de Eid)")
plt.ylabel("Promedio embarques")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')
df['fecha_sin_hora'] = df['con_fecha'].dt.date

def clasificar_billete(codigo):
    if codigo in [1, 2, 3]:
        return 'Pasajero'
    elif codigo in [0, 5, 6, 7, 8, 9]:
        return 'Vehículo'
    else:
        return 'Otro'
df['categoria'] = df['con_tipobillete'].apply(clasificar_billete)

def preparar_ventanas(df, festivo_col):
    fechas_eid = df.loc[df[festivo_col] == 1, 'fecha_sin_hora'].drop_duplicates()
    todos = []
    for eid_day in fechas_eid:
        eid_dt = pd.to_datetime(eid_day)
        ventana = df[(df['con_fecha'] >= eid_dt - pd.Timedelta(days=7)) &
                     (df['con_fecha'] <= eid_dt + pd.Timedelta(days=7))].copy()
        ventana['dias_relativos'] = (ventana['con_fecha'] - eid_dt).dt.days
        ventana['eid_date'] = eid_dt
        conteo = ventana.groupby(['dias_relativos','categoria']).size().reset_index(name='num_embarques')
        conteo['eid_date'] = eid_dt
        todos.append(conteo)
    return pd.concat(todos, ignore_index=True)

def plot_comparativo(df_consolidado, titulo, year=2024):
    df_global = df_consolidado.groupby(['dias_relativos', 'categoria'])['num_embarques'].mean().reset_index()
    pivot_global = df_global.pivot(index='dias_relativos', columns='categoria', values='num_embarques').fillna(0)

    df_2024 = df_consolidado[df_consolidado['eid_date'].dt.year == year]
    df_avg_2024 = df_2024.groupby(['dias_relativos', 'categoria'])['num_embarques'].mean().reset_index()
    pivot_2024 = df_avg_2024.pivot(index='dias_relativos', columns='categoria', values='num_embarques').fillna(0)

    plt.figure(figsize=(10,6))
    for cat in pivot_global.columns:
        plt.plot(pivot_global.index, pivot_global[cat], marker='o', linestyle='-', label=f"Global - {cat}")
    for cat in pivot_2024.columns:
        plt.plot(pivot_2024.index, pivot_2024[cat], marker='o', linestyle='--', label=f"2024 - {cat}")

    plt.axvline(0, color='red', linestyle='--', label='Día Eid')
    plt.title(titulo)
    plt.xlabel("Dias relativos (0 = dia de Eid)")
    plt.ylabel("Promedio de embarques")
    plt.legend()
    plt.tight_layout()
    plt.show()

df_alfitr = preparar_ventanas(df, festivo_col='is_eid_alfitr')
plot_comparativo(df_alfitr, "Comparacion Global vs 2024  Eid al-Fitr (7 dias)")

df_aladha = preparar_ventanas(df, festivo_col='is_eid_aladha')
plot_comparativo(df_aladha, "Comparacion Global vs 2024  Eid al-Adha (7 dias)")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df_filtrado = df[df['porc_ocupacion_vehiculos'].notna()]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.boxplot(x=df_filtrado['porc_ocupacion_vehiculos'])
plt.title("Boxplot Porcentaje de ocupación de vehiculos")

plt.subplot(1, 2, 2)
sns.histplot(df_filtrado['porc_ocupacion_vehiculos'], bins=50, kde=True)
plt.title("Distribucion Porcentaje de ocupación de vehiculos")

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df_copy = df.copy()

resumen_operativas = df_copy.groupby(['con_bacoope', 'buq_nombre']).agg({
    'porc_ocupacion_pasajeros': 'max',
    'porc_ocupacion_vehiculos': 'max'
}).reset_index()

operativas_sobre_pasajeros = resumen_operativas[resumen_operativas['porc_ocupacion_pasajeros'] > 100].copy()
operativas_sobre_vehiculos = resumen_operativas[resumen_operativas['porc_ocupacion_vehiculos'] > 100].copy()

operativas_sobre_pasajeros['exceso_pasajeros'] = (operativas_sobre_pasajeros['porc_ocupacion_pasajeros'] - 100).round(1)
operativas_sobre_vehiculos['exceso_vehiculos'] = (operativas_sobre_vehiculos['porc_ocupacion_vehiculos'] - 100).round(1)

summary_pasajeros = operativas_sobre_pasajeros.groupby('buq_nombre').agg(
    exceso_pasajeros=('exceso_pasajeros', lambda x: [f"{v}%" for v in x])
).reset_index()

summary_vehiculos = operativas_sobre_vehiculos.groupby('buq_nombre').agg(
    exceso_vehiculos=('exceso_vehiculos', lambda x: [f"{v}%" for v in x])
).reset_index()

summary_total = pd.merge(summary_pasajeros, summary_vehiculos, on='buq_nombre', how='outer')

def media_excesos(lista):
    if not isinstance(lista, list) or not lista:
        return 0
    valores = [float(v.replace('%', '')) for v in lista]
    return np.mean(valores)

summary_total['exceso_medio_pasajeros'] = summary_total['exceso_pasajeros'].apply(media_excesos)
summary_total['exceso_medio_vehiculos'] = summary_total['exceso_vehiculos'].apply(media_excesos)

df_exceso = summary_total[
    (summary_total['exceso_medio_pasajeros'] > 0) | (summary_total['exceso_medio_vehiculos'] > 0)
].copy()

df_exceso = df_exceso.sort_values(by=['exceso_medio_pasajeros', 'exceso_medio_vehiculos'], ascending=False)

plt.figure(figsize=(12,6))
bar_width = 0.4
x = range(len(df_exceso))

plt.bar(x, df_exceso['exceso_medio_pasajeros'], width=bar_width, label='Pasajeros', color='steelblue')
plt.bar([i + bar_width for i in x], df_exceso['exceso_medio_vehiculos'], width=bar_width, label='Vehículos', color='darkorange')

plt.xticks([i + bar_width / 2 for i in x], df_exceso['buq_nombre'], rotation=45, ha='right')
plt.ylabel('Exceso medio de ocupacion (%)')
plt.title('Exceso medio (%) por buque')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

limite = 120 

df_limited = df[(df['diferencia_min'] >= -limite) & (df['diferencia_min'] <= limite)]

plt.figure(figsize=(10,5))
sns.histplot(df_limited['diferencia_min'], bins=100, kde=True)
plt.axvline(0, color='red', linestyle='--', label='Hora de zarpe')
plt.title("Distribucion 2h - diferencia minutos")
plt.xlabel("Minutos desde la hora de zarpe")
plt.ylabel("Nº tarjetas leidas")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')
df['anio'] = df['con_fecha'].dt.year
df['hora'] = df['con_fecha'].dt.hour

df_horas = df.groupby(['anio', 'hora']).size().reset_index(name='num_registros')

plt.figure(figsize=(10,6))
for year in sorted(df_horas['anio'].unique()):
    data_year = df_horas[df_horas['anio'] == year]
    plt.plot(data_year['hora'], data_year['num_registros'], marker='o', label=f"Año {year}")

plt.title("Distribucion lecturas por hora del dia (año)")
plt.xlabel("Hora del dia")
plt.ylabel("Nº lecturas")
plt.xticks(range(24))
plt.legend(title='Año')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

cols_interes = [
    'total_pasajeros',
    'total_vehiculos',
    'capacidad_pasajeros',
    'capacidad_vehiculos',
    'porc_ocupacion_pasajeros',
    'porc_ocupacion_vehiculos',
    'diferencia_min',
    'anio',
    'month',
    'week_of_year',
    'day_of_week',
    'season',
    'is_weekend'
]

df_corr = df[cols_interes].copy()

matriz_corr = df_corr.corr()

plt.figure(figsize=(12,10))
sns.heatmap(matriz_corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Matriz de correlacion de variables relevantes")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

cols_interes = [
    'total_pasajeros',
    'total_vehiculos',
    'porc_ocupacion_pasajeros',
    'porc_ocupacion_vehiculos',
    'diferencia_min',
    'anio',
    'month',
    'day_of_week',
    'is_weekend',
    'is_festivo_nacional',
    'is_festivo_local',
    'is_eid_aladha',
    'is_eid_aladha_prev',
    'is_eid_aladha_post',
    'is_eid_alfitr',
    'is_eid_alfitr_prev',
    'is_eid_alfitr_post',
    'is_mawlid_nabi'
]

df_corr = df[cols_interes].copy()

matriz_corr = df_corr.corr()

plt.figure(figsize=(12,10))
sns.heatmap(matriz_corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Matriz de correlación de variables relevantes (incluyendo festivos)")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np

df_sample = df.sample(n=10000, random_state=42)

y = df_sample['total_pasajeros']

predictor_cols = [
    'con_tipobillete',         
    'capacidad_pasajeros',       
    'capacidad_vehiculos',      
    'diferencia_min',           
    'total_vehiculos',           
    'porc_ocupacion_pasajeros',  
    'porc_ocupacion_vehiculos', 
    'day_of_week',               
    'is_weekend',          
    'month',                   
    'week_of_year',              
    'season',                    
    'is_festivo_nacional',       
    'is_festivo_local',          
    'is_eid_aladha',             
    'is_eid_aladha_prev',        
    'is_eid_aladha_post',        
    'is_eid_alfitr',            
    'is_eid_alfitr_prev',        
    'is_eid_alfitr_post',        
    'is_mawlid_nabi',           
    'anio'                     
]

X = df_sample[predictor_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

importancia = pd.DataFrame({
    'Variable': X.columns,
    'Importancia': model.feature_importances_
}).sort_values(by='Importancia', ascending=False)

print("Importancia de las variables predictoras en la muestra:")
display(importancia)

plt.figure(figsize=(10,6))
plt.barh(importancia['Variable'], importancia['Importancia'], color='skyblue')
plt.gca().invert_yaxis()  # La variable más importante en la parte superior
plt.title("Importancia de variables para predecir total_pasajeros (muestra)")
plt.xlabel("Importancia")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split

df_sample = df.sample(n=10000, random_state=42)

y = df_sample['total_pasajeros']

predictor_cols = [
    'con_tipobillete',          
    'capacidad_pasajeros',       
    'capacidad_vehiculos',       
    'diferencia_min',            
    'total_vehiculos',           
    'porc_ocupacion_pasajeros', 
    'porc_ocupacion_vehiculos',  
    'day_of_week',              
    'is_weekend',                
    'month',                     
    'week_of_year',             
    'season',                    
    'is_festivo_nacional',       
    'is_festivo_local',          
    'is_eid_aladha',          
    'is_eid_aladha_prev',       
    'is_eid_aladha_post',        
    'is_eid_alfitr',             
    'is_eid_alfitr_prev',       
    'is_eid_alfitr_post',        
    'is_mawlid_nabi',            
    'anio'                     
]

X = df_sample[predictor_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model_xgb = xgb.XGBRegressor(n_estimators=100, random_state=42)
model_xgb.fit(X_train, y_train)

importancia = pd.DataFrame({
    'Variable': X.columns,
    'Importancia': model_xgb.feature_importances_
}).sort_values(by='Importancia', ascending=False)

print("Importancia de las variables predictoras con XGBoost en la muestra:")
display(importancia)

plt.figure(figsize=(10,6))
plt.barh(importancia['Variable'], importancia['Importancia'], color='lightgreen')
plt.gca().invert_yaxis() 
plt.title("Importancia de variables para predecir total_pasajeros (XGBoost, muestra)")
plt.xlabel("Importancia")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')

df['hora'] = df['con_fecha'].dt.hour

df_grouped = df.groupby(['con_tipobillete', 'hora']).size().reset_index(name='num_registros')

pivot_table = df_grouped.pivot(index='hora', columns='con_tipobillete', values='num_registros').fillna(0)
print(pivot_table)

plt.figure(figsize=(12,6))
sns.heatmap(pivot_table, annot=True, fmt=".0f", cmap='viridis')
plt.title("Afluencia por tipo de billete y hora")
plt.xlabel("Tipo de billete")
plt.ylabel("Hora")
plt.tight_layout()
plt.show()

In [ ]:
display(df.head())
df.info()

In [ ]:
columnas_a_excluir = [
    'con_clave',
    'con_bacoope',
    'con_tipo',
    'con_acredita',
    'con_nombre',
    'con_codigo',
    'con_naviera',
    'con_dni',
    'con_cupon',
    'con_trayecto',
    'con_shortcomp',
    'buq_nombre',
    'anio',
    'anio_con_fecha',
    'anio_con_dateticket',
    'agrupacion_billete',
    'fecha_mensual',
    'hora',
    'nombre_dia',
    'tipo_billete_clasificado'

]

df_modelo = df.drop(columns=columnas_a_excluir).copy()

In [36]:
df_modelo = df_modelo.rename(columns={'fecha_sin_hora': 'fecha_embarque'})
df_modelo = df_modelo.rename(columns={'categoria': 'agrupacion'})

In [ ]:
df_modelo['con_fecha'] = pd.to_datetime(df['con_fecha'], errors='coerce')

df_modelo['dia_embarque'] = df_modelo['con_fecha'].dt.dayofweek + 1
df_modelo['dia_embarque'] = df_modelo['dia_embarque'].astype('int64')

valores_unicos = df_modelo['dia_embarque'].unique()
print("Valores unicos en 'dia_embarque':", valores_unicos)

In [ ]:

df_modelo['tipo_agrupacion'] = df_modelo['agrupacion'].apply(lambda x: 1 if x == 'Pasajero' else 0)

df_modelo.drop(columns=['agrupacion'], inplace=True)

In [ ]:
print(df_modelo['tipo_agrupacion'].value_counts())

In [ ]:

df_modelo['con_fecha'] = pd.to_datetime(df_modelo['con_fecha'], errors='coerce')

df_modelo['mes_embarque'] = df_modelo['con_fecha'].dt.month

print(df_modelo['mes_embarque'].unique())


In [41]:
df_modelo.drop(columns=['day_of_week'], inplace=True)
df_modelo.drop(columns=['month'], inplace=True)

In [ ]:
df_modelo['fecha_embarque'] = pd.to_datetime(df_modelo['fecha_embarque'], errors='coerce')

In [ ]:
display(df_modelo.sample(n=10))

df_modelo.info()

CONTINUACION

In [ ]:
df_modelo = pd.get_dummies(df_modelo, columns=['anio_embarque'], prefix='anio')

print(df_modelo.columns)

In [ ]:
one_hot_cols = ['anio_2022', 'anio_2023', 'anio_2024']
df_modelo[one_hot_cols] = df_modelo[one_hot_cols].astype(int)

In [48]:
df_modelo['dia_del_anio'] = df_modelo['fecha_embarque'].dt.dayofyear

In [ ]:
one_hot_season = pd.get_dummies(df_modelo['season'], prefix='season').astype(int)

df_modelo = pd.concat([df_modelo, one_hot_season], axis=1)

In [ ]:
display(df_modelo.sample(n=5))

df_modelo.info()

# **CREAR COPIA DEL DATASET:**

In [ ]:

df_final = df_modelo.copy()

In [ ]:

dias_unicos = df_final['dia_del_anio'].unique()
print("Valores únicos en 'dia_del_anio':", sorted(dias_unicos))
print("¿Incluye el día 366?", 366 in dias_unicos)

In [ ]:

df_final['dia_del_anio_sin'] = np.sin(2 * np.pi * df_final['dia_del_anio'] / 366)
df_final['dia_del_anio_cos'] = np.cos(2 * np.pi * df_final['dia_del_anio'] / 366)

df_final['dia_embarque_sin'] = np.sin(2 * np.pi * df_final['dia_embarque'] / 7)
df_final['dia_embarque_cos'] = np.cos(2 * np.pi * df_final['dia_embarque'] / 7)

df_final['hora_embarque_sin'] = np.sin(2 * np.pi * df_final['hora_embarque'] / 24)
df_final['hora_embarque_cos'] = np.cos(2 * np.pi * df_final['hora_embarque'] / 24)

df_final['mes_embarque_sin'] = np.sin(2 * np.pi * df_final['mes_embarque'] / 12)
df_final['mes_embarque_cos'] = np.cos(2 * np.pi * df_final['mes_embarque'] / 12)

df_final['week_of_year_sin'] = np.sin(2 * np.pi * df_final['week_of_year'] / 53)
df_final['week_of_year_cos'] = np.cos(2 * np.pi * df_final['week_of_year'] / 53)


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(12, 10))

axs[0, 0].scatter(df_final['dia_del_anio_sin'], df_final['dia_del_anio_cos'], alpha=0.5)
axs[0, 0].set_title('Codificación Cíclica: Día del Año')
axs[0, 0].set_xlabel('sin(dia_del_anio)')
axs[0, 0].set_ylabel('cos(dia_del_anio)')
axs[0, 0].axis('equal')
axs[0, 0].grid(True)

axs[0, 1].scatter(df_final['dia_embarque_sin'], df_final['dia_embarque_cos'], alpha=0.5)
axs[0, 1].set_title('Codificación Cíclica: Día de la Semana')
axs[0, 1].set_xlabel('sin(dia_embarque)')
axs[0, 1].set_ylabel('cos(dia_embarque)')
axs[0, 1].axis('equal')
axs[0, 1].grid(True)

axs[1, 0].scatter(df_final['mes_embarque_sin'], df_final['mes_embarque_cos'], alpha=0.5)
axs[1, 0].set_title('Codificación Cíclica: Mes del Año')
axs[1, 0].set_xlabel('sin(mes_embarque)')
axs[1, 0].set_ylabel('cos(mes_embarque)')
axs[1, 0].axis('equal')
axs[1, 0].grid(True)

axs[1, 1].scatter(df_final['week_of_year_sin'], df_final['week_of_year_cos'], alpha=0.5)
axs[1, 1].set_title('Codificación Cíclica: Semana del Año')
axs[1, 1].set_xlabel('sin(week_of_year)')
axs[1, 1].set_ylabel('cos(week_of_year)')
axs[1, 1].axis('equal')
axs[1, 1].grid(True)

plt.tight_layout()
plt.show()



# Con estos dos bloques pruebo a crear variables del dia de la semana. Ejecutar solo si con este nuevo dataset mejora el rendimiento del modelo, sino guardar el dataset anterior

In [ ]:
display(df.sample(n=5))
df_final.info()

In [ ]:

df_final['weekday'] = df_final['con_fecha'].dt.weekday

for i in range(7):
    day_names = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    df_final[f'is_{day_names[i]}'] = (df_final['weekday'] == i).astype(int)
df_final = df_final.drop(columns=['weekday'])

In [ ]:

df_final['weekday'] = df_final['con_fecha'].dt.weekday

df_final['weekday_sin'] = np.sin(2 * np.pi * df_final['weekday']/7)
df_final['weekday_cos'] = np.cos(2 * np.pi * df_final['weekday']/7)

day_names = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
for i in range(7):
    df_final[f'is_{day_names[i]}'] = (df_final['weekday'] == i).astype(int)

df_final = df_final.drop(columns=['weekday'])

In [ ]:
display(df_final.sample(n=5))
df_final.info()

In [ ]:

df_final.to_csv("dataset_final_convariablesdedias.csv", index=False)

from google.colab import files
files.download("dataset_final_convariablesdedias.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>